In [39]:
import pandas as pd
import numpy as np
from collections import Counter
from operator import itemgetter

Leemos el csv que creamos previamente

In [38]:
df_Films = pd.read_csv('Datasets/Datasets_creados/Films.csv')
df_Films

,Unnamed: 0,Id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Plataform
0,0,1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,No info,113 min,"Comedy, Drama",A small fishing village must procure a local d...,Amazon Prime
1,1,2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,Amazon Prime
2,2,3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,No info,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,Amazon Prime
3,3,4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,No info,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",Amazon Prime
4,4,5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,No info,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,Amazon Prime
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22993,8802,22994,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a...",Netflix
22994,8803,22995,TV Show,Zombie Dumb,No info,No info,No info,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g...",Netflix
22995,8804,22996,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,Netflix
22996,8805,22997,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...",Netflix


In [4]:
Amazon =df_Films[df_Films['Plataform'] == 'Amazon Prime']
Disney =df_Films[df_Films['Plataform'] == 'Disney Plus']
Netflix =df_Films[df_Films['Plataform'] == 'Netflix']
Hulu =df_Films[df_Films['Plataform'] == 'Hulu']

Creamos las Querys que pide el Proyecto final, definiendo funciones que luego exportaremos. 

Máxima duración según tipo de film (película/serie), por plataforma y por año

In [80]:
def Max_temp_x_pel_y_año (año, plataforma, tipo):
    #filtro en duracion por los que contienen 'min'
    df_max_temp = df_Films[df_Films.duration.str.contains(tipo)]

    #filtro por plataforma
    df_max_temp = df_max_temp[df_max_temp['Plataform'] == plataforma]

    #filtro por año
    df_max_temp = df_max_temp[df_max_temp['release_year'] == año]

    #quito min de la columna duration
    df_max_temp['duration'] = df_max_temp['duration'].apply(lambda x: x.replace("min",""))
    df_max_temp['duration'] = df_max_temp['duration'].str.strip()

    #cambio el tipo de valores de la columna de objet a int
    df_max_temp['duration'] = df_max_temp['duration'].astype(object).astype(int)

    #reordeno el dataframe
    df_max_temp = df_max_temp.sort_values('duration',ascending=False)
    df_max_temp = pd.DataFrame(df_max_temp)
    df_max_temp = df_max_temp.drop(['Id','director','cast','country','date_added','release_year','rating','duration','listed_in','description','Plataform',	'Unnamed: 0', 'type'], axis = 1)
    
    #filtrom la promera fila
    return df_max_temp.iloc[0,0]

In [81]:
Max_temp_x_pel_y_año (2018, 'Hulu', 'min')

'The House That Jack Built'

In [161]:
def cantidad_film (plataforma):
    cant_film = df_Films[df_Films['Plataform'] == plataforma]
    x =cant_film.groupby(['type']) 
    x = x.drop(['Id','title','director','cast','country','date_added','release_year','rating','duration','listed_in','description','Plataform'], axis = 1)
    x = x.transpose()
    x.insert(2,"Plataform",[plataforma],True)
    #x = x.drop(['type'], axis = 0)

    Movie = x.iloc[0,0]
    TV_Show = x.iloc[0,1]
    plataform = x.iloc[0,2]
    #return print('en la plataforma', plataform ,'hay', TV_Show, 'series y', Movie,'Peliculas')
    return x

In [163]:
cantidad_film('Netflix')

type,Movie,TV Show,Plataform
Unnamed: 0,6131,2676,Netflix


In [8]:
def actor_mas_actuo (plataforma, año):
    actor = df_Films[df_Films['Plataform'] == plataforma]
    actor = actor[actor['release_year'] == año]
    lista_prov = actor.drop(['Id','title','director', 'country','date_added', 'rating','duration','description','type', 'Unnamed: 0'], axis = 1)

    lista_prov = lista_prov[lista_prov['cast'] != 'No info']
    lista_prov = lista_prov['cast'].str.split(', ').to_list()
    lista_actores = []
    for item in lista_prov:

        lista_actores += item

    #Ahora contamos las veces en que aparece un actor y lo metemos en un diccionario
    contador = Counter(lista_actores)

    #ordenamos el diccionario de forma descendente
    contador = sorted(contador.items(), key = itemgetter (1), reverse = True)

    mas_actuo = contador[0]
    mas_actuo = list(mas_actuo)
    
    df_actor = pd.DataFrame()
    actores = mas_actuo[0]
    cantidad = mas_actuo[1]
    df_actor['plataform'] = None
    df_actor['actores'] = None
    df_actor['cantidad'] = None
    

    nueva_fila = pd.Series([plataforma, actores, cantidad], index=df_actor.columns) 
    df_actor = df_actor.append(nueva_fila, ignore_index=True)


    return df_actor

In [9]:
actor_mas_actuo('Netflix', 2018)

C:\Users\elune\AppData\Local\Temp\ipykernel_16840\2099279289.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_actor = df_actor.append(nueva_fila, ignore_index=True)


,plataform,actores,cantidad
0,Netflix,Andrea Libman,8


Maxima duracion de una Serie y cual Maxima duracion de una pelicula y cual *falta y crear una union entre los filtros y el resultado*

TVSerie = TVSerie[TVSerie['duration'] == TVSeries_max]
TVShow_max = TVSerie['title']
#Pido el valor maximo solo para que me devuelva el resultado como una variable, ya que el DatFrame solo se compone de una fila.
TVShow_max = TVShow_max.max()
TVShow_max

Cantidad de películas y series (separado) por plataforma

In [158]:
def cantidad_film (plataforma:str):
    cant_film = df_Films[df_Films['Plataform'] == plataforma]
    x =cant_film.groupby(['type']).count() 
    x = x.drop(['Id','title','director','cast','country','date_added','release_year','rating','duration','listed_in','description','Plataform'], axis = 1)
    x = x.transpose()
    x.insert(2,"Plataform",[plataforma],True)
    #x = x.drop(['type'], axis = 0)
    return x

In [159]:
#Funcion de preuba
cantidad_film ('Amazon Prime')

type,Movie,TV Show,Plataform
Unnamed: 0,7814,1854,Amazon Prime


Cantidad de veces que se repite un género y plataforma con mayor frecuencia del mismo

In [170]:
def top_genero (genero):
    top_genero = df_Films[df_Films.listed_in.str.contains(genero)]
    contando = top_genero['Plataform'].value_counts()
    contando = pd.DataFrame(contando)
    resultado = contando.iloc[0,0]
    return contando.head(1)


In [171]:
top_genero('Comedy')

,Plataform
Amazon Prime,2099


Actor que más se repite según plataforma y año

In [12]:
def Actor_mas_actuo(genero, plataforma):

    #filtro para las plataformas
    Amazon =df_Films[df_Films['Plataform'] == 'Amazon Prime']
    Disney =df_Films[df_Films['Plataform'] == 'Disney Plus']
    Netflix =df_Films[df_Films['Plataform'] == 'Netflix']
    Hulu =df_Films[df_Films['Plataform'] == 'Hulu']

    #filtro las plataformas con if
    if plataforma == 'Amazon_Prime':
        x = Amazon.copy() 

    if plataforma == 'Disney_Plus':
        x = Disney.copy()

    if plataforma == 'Netflix':
        x = Netflix.copy() 

    if plataforma == 'Hulu':
        x = Hulu.copy()  

    df_prueba = x[x['listed_in'].str.contains(genero)]
    return df_prueba.shape[0]

In [13]:
#Funcion de prueba
Actor_mas_actuo('Comedy','Amazon_Prime')

2099

TOP 5 actores con más films actuados

In [1]:
def TOP_actores():
    TOP_actores = df_Films[df_Films['cast'] != 'No info']

        #cambiamos el tipo de columna para poder utilizar la funcion split
    TOP_actores['cast'] = TOP_actores['cast'].astype(object).astype(str)

        #utilizamos la funcion split para poder separar los actores y meterlos en una lista
    lista_actores_en_lista = TOP_actores['cast'].str.split(', '). to_list()
    lista_actores = []
    for item in lista_actores_en_lista:

        lista_actores += item

    #Ahora contamos las veces en que aparece un actor y lo metemos en un diccionario
    contador = Counter(lista_actores)

    #ordenamos el diccionario de forma descendente
    contador = sorted(contador.items(), key = itemgetter (1), reverse = True)
    
    #Por ultimo pedimos solo los primeros 5
    return contador

    

In [2]:
#funcionj de prueba
TOP_actores()

NameError: name 'df_Films' is not defined

TOP 5 directores con mas filma dirigidos

In [364]:
def TOP_directores():
    TOP_directores = df_Films[df_Films['director'] != 'No info']

    #cambiamos el tipo de columna para poder utilizar la funcion split
    TOP_directores['director'] = TOP_directores['director'].astype(object).astype(str)

    #creamos una lista con los directores
    lista_directores = TOP_directores['director'].to_list()
    lista_directores

    #En este caso no hay necesidad de el paso para crear una sola lista ya que es una sola desde el princpio
    #Ahora contamos las veces en que aparece un director y lo metemos en un diccionario
    contador = Counter(lista_directores)

    #ordenamos el diccionario de forma descendente
    contador = sorted(contador.items(), key = itemgetter (1), reverse = True)
    
    #Por ultimo pedimos solo los primeros 5
    return contador[0:5]

In [365]:
#funcion de prueba
TOP_directores()

C:\Users\elune\AppData\Local\Temp\ipykernel_16192\3047744473.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TOP_directores['director'] = TOP_directores['director'].astype(object).astype(str)


[('Mark Knight', 113),
 ('Cannis Holder', 61),
 ('Jay Chapman', 46),
 ('Moonbug Entertainment', 37),
 ('Arthur van Merwijk', 30)]